<a href="https://colab.research.google.com/github/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal_Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classificação de Saúde Fetal - Pycaret**

<p align=center>
<img src="https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/baby.jpg?raw=true" width="50%"></p>
<p align=center>Fonte: Freepik

Esse artigo abordará somente o auto machine learning através da bilioteca Pycaret. Para mais informações sobre a análise de dados e business understanding verifique esse link: 

https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal.ipynb

A biblioteca Pycaret permite procurar e selecionar o melhor modelo que terá desempenho satisfatório para o nosso dataset. Primerio as bibliotecas serão instaladas e os dados serão carregados. Em seguida importaremos o módulo de classificação, para depois fazer a divisão do dataset entre treino e teste. 

Para achar o melhor modelo, primeiro será feita uma busca com `setup` para obtermos um rank dos melhores modelos e suas métricas. Em seguida selecionaremos o melhor e então fazemos o `tunning` dos seus hiper-parâmetros com posterior análise através de `evaluate_model`. Após essas etapas, é feito predições dos dados de teste com `predict` e então o modelo é finalizado e salvo.

In [1]:
# Importação das bibliotecas necessárias

import pandas as pd

In [2]:
data = 'https://raw.githubusercontent.com/Diego875621/Data_Science/main/Fetal%20Health%20Classification/fetal_health.csv'

df = pd.read_csv(data)
df.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [3]:
# Instalação pycaret
!pip install pycaret -q

     |████████████████████████████████| 320 kB 4.7 MB/s 
     |████████████████████████████████| 1.3 MB 53.6 MB/s 
     |████████████████████████████████| 25.9 MB 1.7 MB/s 
     |████████████████████████████████| 88 kB 6.8 MB/s 
     |████████████████████████████████| 6.8 MB 16.1 MB/s 
     |████████████████████████████████| 3.3 MB 36.2 MB/s 
     |████████████████████████████████| 17.0 MB 39.1 MB/s 
     |████████████████████████████████| 315 kB 46.5 MB/s 
     |████████████████████████████████| 636 kB 53.4 MB/s 
     |████████████████████████████████| 4.8 MB 29.6 MB/s 
     |████████████████████████████████| 2.0 MB 42.7 MB/s 
     |████████████████████████████████| 1.7 MB 47.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 141 kB 63.3 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |█████████████████

In [4]:
# Desabilitar mensagens de log
#import logging, sys
#logging.disable(sys.maxsize)

# Habilitar o Colab mode
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


In [6]:
# Módulos de classificação]
import numpy as np
from pycaret.classification import *

In [7]:
# Divisão treino x test
test = df.sample(frac = 0.25)
train = df.drop(test.index)

test.reset_index(inplace = True, drop = True)
train.reset_index(inplace = True, drop = True)

In [8]:
# Setup
clf = setup(data = df, target = 'fetal_health', fix_imbalance=True, remove_outliers=True,
            log_experiment = True, experiment_name = 'Reg01')

,Description,Value
0,session_id,5075
1,Target,fetal_health
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(2126, 22)"
5,Missing Values,False
6,Numeric Features,20
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:Logging experiment in MLFlow
2022/11/08 21:15:46 INFO mlflow.tracking.fluent: Experiment with name 'Reg01' does not exist. Creating a new experiment.
INFO:logs:SubProcess save_model() called ==================================
INFO:logs:Initializing save_model()
INFO:logs:save_model(model=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='fetal_health',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical

PicklingError: ignored

In [9]:
# Comparação entre os modelos
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9483,0.9845,0.8976,0.9500,0.9480,0.8473,0.8497,1.223
gbc,Gradient Boosting Classifier,0.9384,0.9816,0.9013,0.9414,0.9388,0.8216,0.8239,2.956
rf,Random Forest Classifier,0.9356,0.9820,0.8730,0.9366,0.9351,0.8096,0.8113,1.077
et,Extra Trees Classifier,0.9307,0.9758,0.8417,0.9314,0.9289,0.7892,0.7929,0.577
dt,Decision Tree Classifier,0.9172,0.8980,0.8524,0.9214,0.9176,0.7600,0.7620,0.100
knn,K Neighbors Classifier,0.8698,0.9261,0.8430,0.8991,0.8783,0.6641,0.6777,0.145
ada,Ada Boost Classifier,0.8592,0.8816,0.8021,0.8805,0.8649,0.6142,0.6234,0.344
lda,Linear Discriminant Analysis,0.8273,0.9543,0.8247,0.8920,0.8452,0.5964,0.6280,0.039
svm,SVM - Linear Kernel,0.8141,0.0000,0.7442,0.8595,0.8213,0.5327,0.5556,0.180
lr,Logistic Regression,0.8125,0.9223,0.8039,0.8714,0.8298,0.5538,0.5790,1.792


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5075, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:compare_models() succesfully completed......................................


In [10]:
# Melhor modelo
best

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5075, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [11]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9225,0.9784,0.9029,0.9247,0.9233,0.7796,0.7802
1,0.9577,0.9780,0.8532,0.9599,0.9558,0.8650,0.8729
2,0.9648,0.9940,0.9336,0.9665,0.9653,0.8996,0.9004
3,0.9433,0.9839,0.8554,0.9415,0.9404,0.8172,0.8227
4,0.9645,0.9944,0.9441,0.9646,0.9643,0.8941,0.8943
5,0.9858,0.9957,0.9812,0.9858,0.9858,0.9587,0.9587
6,0.9220,0.9829,0.8641,0.9309,0.9247,0.7872,0.7913
7,0.9716,0.9975,0.9365,0.9723,0.9716,0.9173,0.9174
8,0.9433,0.9744,0.8343,0.9411,0.9410,0.8253,0.8284


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 3
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5075, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................


In [12]:
# Tunning
tuned_lightgbm = tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8944,0.9691,0.8911,0.9093,0.8993,0.7183,0.7245
1,0.9296,0.9670,0.8155,0.9291,0.9265,0.7750,0.7821
2,0.9296,0.9927,0.8930,0.9363,0.9319,0.8051,0.8075
3,0.9007,0.9679,0.8514,0.9058,0.9027,0.7155,0.7168
4,0.9433,0.9949,0.9490,0.9504,0.9452,0.8420,0.8460
5,0.9574,0.9971,0.9435,0.9640,0.9587,0.8822,0.8857
6,0.9220,0.9866,0.9157,0.9390,0.9263,0.7948,0.8048
7,0.9645,0.9947,0.9722,0.9670,0.9652,0.9010,0.9025
8,0.9007,0.9564,0.8294,0.9109,0.9041,0.7252,0.7281


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 4
INFO:logs:LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:tune_model() succesfully completed......................................


In [13]:
# Comparação dos hiper-parâmetros antes e pós tunning
print(lightgbm)

print('\n\n')

print(' GBR após Tunning: \n', tuned_lightgbm)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=5075, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)



 GBR após Tunning: 
 LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, su

In [14]:
# Avaliação do modelo
evaluate_model(tuned_lightgbm)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [15]:
final_lightgbm = finalize_model(tuned_lightgbm)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), fit_kwargs=None, groups=None, model_only=True, display=None, experiment_custom_tags=None, return_train_score=False)
INFO:logs:Finalizing LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_sampl

In [16]:
final_lightgbm

LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [17]:
predict_model(final_lightgbm)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9639,0.994,0.9587,0.9658,0.9645,0.9023,0.903


IndexError: ignored

In [18]:
test_predict = predict_model(final_lightgbm, data = test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9605,0.9928,0.9467,0.9619,0.9611,0.893,0.8933


IndexError: ignored

In [19]:
save_model(final_lightgbm, 'lightgbm_classifier_06_11_2022')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LGBMClassifier(bagging_fraction=0.5, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=36, min_child_weight=0.001, min_split_gain=0.8,
               n_estimators=40, n_jobs=-1, num_leaves=50, objective=None,
               random_state=5075, reg_alpha=4, reg_lambda=1, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), model_name=lightgbm_classifier_06_11_2022, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      num

PicklingError: ignored

O modelo selecionado pelo Pycaret como melhor foi o LightGBM, atingindo as seguintes métricas: 
- F1 score = 0.9467
- AUC = 0.9605
- Kappa score = 0.893

Para ter acesso a discussões e conclusões da análise desse dataset, acesse:

https://github.com/Diego875621/Data_Science/blob/main/Fetal%20Health%20Classification/Classifica%C3%A7%C3%A3o_de_Sa%C3%BAde_Fetal.ipynb

Imagem:

<a href="https://www.freepik.com/free-vector/baby-birth-concept-illustration_21532448.htm#query=fetus&position=12&from_view=search&track=sph">Image by storyset</a> on Freepik